# Importing Data & Packages

In [1]:
#pip install pandas

In [2]:
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import random

print('Everything Imported Succesfully!')

Everything Imported Succesfully!


In [3]:
data=pd.read_csv('/kaggle/input/301-330-days/Data_5IAL_3_301.csv',index_col=0,parse_dates=[1])
data.rename(columns={'0':'Date'},inplace=True)
data['5IAL_3_301.BatchName']=data['5IAL_3_301.BatchName'].fillna('No Batch Specified')
data.head()

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3553: DtypeWarning: Columns (14) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Date,5IAL_3_TT301.50,5IAL_3_QIT301.52,5IAL_3_PIT 301.55,5IAL_3_QIT301.57,5IAL_3_PIT301.60,5IAL_3_FIT301.61MF,5IAL_3_FIT301.61VF,5IAL_3_FIT301.61D,5IAL_3_PIT301.63,...,5IAL_3_GSC301.44,5IAL_3_GSO301.44,5IAL_3_LSL301.64,5IAL_3_LSL301.68,5IAL_3_LSLL301.69,5IAL_3_301.OCCUPIED,5IAL_3_LIT301.54,5IAL_3_LSH301.56,5IAL_3_XPV301.13,5IAL_3_WY301.54
0,2022-04-10 12:00:00,21.80,9.84,-0.02,0.0,0.0,0.0,0.0,0.00105,2.07,...,0,0,0,0,0,0,1.40,0,0.0,0.0
1,2022-04-10 12:01:00,21.80,9.84,-0.02,0.0,0.0,0.0,0.0,0.00106,2.07,...,0,0,0,0,0,0,1.40,0,0.0,0.0
2,2022-04-10 12:02:00,21.80,9.84,-0.02,0.0,0.0,0.0,0.0,0.00102,2.07,...,0,0,0,0,0,0,1.41,0,0.0,0.0
3,2022-04-10 12:03:00,21.78,9.84,-0.02,0.0,0.0,0.0,0.0,0.00105,2.07,...,0,0,0,0,0,0,1.40,0,0.0,0.0
4,2022-04-10 12:04:00,21.78,9.84,-0.02,0.0,0.0,0.0,0.0,0.00104,2.07,...,0,0,0,0,0,0,1.40,0,0.0,0.0


# Data Splitting and Grouping 

In [4]:
Batch_Names=data['5IAL_3_301.BatchName']
unique_names=Batch_Names.unique()
print(unique_names)

['No Batch Specified' 'YP5812210101' 'YP5812210102' ... 'KB212306501'
 'NA KB212306501' 'KC112306501']


In [5]:
print(unique_names.size)

1629


In [6]:
#Convert the unique_names numpy array to a pandas series
temp_df=pd.Series(unique_names)

# Create a boolean mask to select rows where the length of the specified column is between 11 and 12 characters (inclusive)
mask = (temp_df.str.len() >= 11) & (temp_df.str.len() <= 12)

# Use the boolean mask to select only the rows that meet the criteria
Viable_Batches = unique_names[mask]

print(Viable_Batches.size)

del temp_df

1205


## Train Test Split

Randomly pick the Test_Train_split fraction of data from the filterd unique name list.\
This way a percentage of batches are not used in training and thus can be used for testing.

In [7]:
To_split=Viable_Batches
Test_Train_split=0.2

Test_Batches=random.sample(To_split.tolist(), round(To_split.size*Test_Train_split))

Train_Batches = [x for x in To_split if x not in Test_Batches]

print(len(Test_Batches))
print(len(Train_Batches))

241
964


In [8]:
Train_Data=data[data['5IAL_3_301.BatchName'].isin(Train_Batches)]
Test_Data=data[data['5IAL_3_301.BatchName'].isin(Test_Batches)]

# Train_Grouped=Train_Data.groupby(["5IAL_3_301.BatchName"],as_index=True)
# Test_Grouped=Test_Data.groupby(["5IAL_3_301.BatchName"],as_index=True)

# Train_Grouped.size()
# Test_Grouped.size()

In [9]:
Train_Data.head()

,Date,5IAL_3_TT301.50,5IAL_3_QIT301.52,5IAL_3_PIT 301.55,5IAL_3_QIT301.57,5IAL_3_PIT301.60,5IAL_3_FIT301.61MF,5IAL_3_FIT301.61VF,5IAL_3_FIT301.61D,5IAL_3_PIT301.63,...,5IAL_3_GSC301.44,5IAL_3_GSO301.44,5IAL_3_LSL301.64,5IAL_3_LSL301.68,5IAL_3_LSLL301.69,5IAL_3_301.OCCUPIED,5IAL_3_LIT301.54,5IAL_3_LSH301.56,5IAL_3_XPV301.13,5IAL_3_WY301.54
1169,2022-04-11 07:29:00,21.75,10.07,-0.02,0.00,0.00,0.00,0.00,0.00128,2.07,...,0,1,0,0,0,0,2.17,0,0.00,0.00
1170,2022-04-11 07:30:00,25.87,10.07,-0.02,0.00,0.01,0.00,0.00,0.00125,2.07,...,0,1,0,0,0,1,6.61,0,0.00,76.62
1171,2022-04-11 07:31:00,25.81,9.90,1.08,133.40,0.01,115.21,6.73,1.02720,2.07,...,1,0,0,0,0,1,22.32,0,2.00,252.31
1172,2022-04-11 07:32:00,26.21,9.92,1.09,167.81,0.01,116.83,6.84,1.02550,2.07,...,1,0,0,0,0,1,32.11,0,1.94,435.71
1173,2022-04-11 07:33:00,26.84,10.09,0.63,187.40,0.00,149.16,8.65,1.03430,2.07,...,1,0,0,0,0,1,32.63,0,1.99,511.36


## Normalise the selection with use of the Z-Score
The Z-score can be described as:\
$Z=\frac{x-\mu}{\sigma}$

In this x is your value, μ is the average of the population that value is taken from, and σ is the standard deviation of the population. It is basicaly a measure of how far a given value is away from its mean in standard deviations. To give you a good feeling, in a normal distribution 66% of values are within 1 std of the mean. 99% are withing 2 std and 99.9 is within 3

Note that i am not yet sure how representative all of this is for the data, it just allows me to use Massflow and resulting pressure more easaly because the range of values are simalar and i dont have to specify two seperate axis with values within 1 figure.

In [10]:
data_norm=pd.DataFrame()
data_norm_values=pd.DataFrame()

df=Train_Data.drop(['Date','5IAL_3_301.BatchName'],axis=1)

data_norm=(df-df.mean())/df.std()

for col in df:
    # zval=(Train_Grouped[col]-np.mean(Train_Grouped[col]))/np.std(Train_Grouped[col])
    # data_norm=pd.concat([data_norm,zval],axis=1)
    data_norm_values[col]=[np.mean(df[col]),np.std(df[col])]



In [11]:
data_norm.head()
#data_norm_values.head()

,5IAL_3_TT301.50,5IAL_3_QIT301.52,5IAL_3_PIT 301.55,5IAL_3_QIT301.57,5IAL_3_PIT301.60,5IAL_3_FIT301.61MF,5IAL_3_FIT301.61VF,5IAL_3_FIT301.61D,5IAL_3_PIT301.63,5IAL_3_P301.70,...,5IAL_3_GSC301.44,5IAL_3_GSO301.44,5IAL_3_LSL301.64,5IAL_3_LSL301.68,5IAL_3_LSLL301.69,5IAL_3_301.OCCUPIED,5IAL_3_LIT301.54,5IAL_3_LSH301.56,5IAL_3_XPV301.13,5IAL_3_WY301.54
1169,-2.049107,0.981643,-1.439621,-0.308810,-0.096873,-1.570852,-0.023770,-3.256979,-0.862856,-1.602790,...,-3.316979,3.318924,-0.012103,-0.102546,-0.041861,-19.042618,-2.027824,NaN,-1.602741,-2.130402
1170,-0.603899,0.981643,-1.439621,-0.308810,7.609359,-1.570852,-0.023770,-3.257078,-0.862856,-1.602790,...,-3.316979,3.318924,-0.012103,-0.102546,-0.041861,0.052513,-1.889844,NaN,-1.602741,-2.011354
1171,-0.624946,0.855463,1.683326,-0.297432,7.609359,0.218670,-0.009873,0.123133,-0.862856,0.660219,...,0.301477,-0.301300,-0.012103,-0.102546,-0.041861,0.052513,-1.401628,NaN,0.628957,-1.738374
1172,-0.484634,0.870308,1.711717,-0.294497,7.609359,0.243833,-0.009646,0.117532,-0.862856,0.649071,...,0.301477,-0.301300,-0.012103,-0.102546,-0.041861,0.052513,-1.097387,NaN,0.562006,-1.453415
1173,-0.263644,0.996488,0.405757,-0.292827,-0.096873,0.746005,-0.005908,0.146526,-0.862856,0.626776,...,0.301477,-0.301300,-0.012103,-0.102546,-0.041861,0.052513,-1.081227,NaN,0.617799,-1.335873
